# Потоки

#### Что такое Thread (Поток)?
Thread – это отдельный поток выполнения. Это означает, что в вашей программе могут работать две и более подпрограммы одновременно. 

Для создания потоков мы будем использовать стандартный модуль threading.

In [1]:
import threading

#### Пример многопоточной программы
Пусть два потока параллельно выводят каждый в свой файл заданное число строк. Для начала нам понадобится функция, которая выполнит задуманный нами сценарий. Аргументами целевой функции будут число строк и имя текстового файла для записи.

In [2]:
def prescript(thefile, num):
    with open(thefile, 'w') as f:
        for i in range(num):
            if num > 500:
                f.write('ABC\n')
            else:
                f.write('abc\n')
thread1 = threading.Thread(target=prescript, args=('f1.txt', 200,))
thread2 = threading.Thread(target=prescript, args=('f2.txt', 1000,))
thread1.start()
thread2.start()
#thread1.join()
#thread2.join()

Метод **start()** запускает ранее созданный поток. Метод **join()** останавливает поток, когда тот выполнит свои задачи и освобождает ресурсы.

##### Thread(group, target, name, args, kwargs)
Здесь **group** - группа потоков (пока не используется, должен быть равен *None*), **target** - объект, который будет вызван в методе **run()**, **name** - имя потока, **args** и **kwargs** - последовательность и словарь позиционных и именованных параметров (соответственно) для вызова заданного в параметре **target** объекта. 

#### Стандартные методы работы с потоками
- current_thread()  — смотрим, какой поток вызвал функцию;
- active_count() — считаем работающие в данный момент экземпляры класса Thread;
- enumerate()   — получаем список работающих потоков.
Ещё можно управлять потоком через методы класса:
- is_alive()  —  спрашиваем поток: «Жив ещё?»  — получаем true или false;
- getName()  — узнаём имя потока;
- setName(any_name)  — даём потоку имя;

У каждого потока, пока он работает, есть уникальный идентификационный номер, который хранится в переменной **ident**.

In [5]:
print(thread1.ident)

18864


##### Timer
Отсрочить операции в вызываемых потоком функциях можно с помощью таймера. В инициализаторе объектов класса **Timer** всего два аргумента — время ожидания в секундах и функция, которую нужно в итоге выполнить:

In [6]:
import threading
print ("Waiting...")
def timer_test():
    print ("The timer has done its job!")
tim = threading.Timer(5.0, timer_test)
tim.start()

Waiting...
The timer has done its job!


#### Простая блокировка в Python
Взаимоисключение (mutual exception, кратко - mutex) - простейшая блокировка, которая на время работы потока с ресурсом закрывает последний от других обращений. Реализуют это с помощью класса **Lock**.

In [7]:
import threading
mutex = threading.Lock()

Мы создали блокировку с именем mutex. Теперь её можно ставить и снимать методами **acquire()** и **release()**:

In [13]:
resource = 0

def thread_safe_function():
    global resource
    for i in range(1000000):
        mutex.acquire()
        # Делаем что-то с переменной resource
        mutex.release()

Обратите внимание: обойти простую блокировку не может даже поток, который её активировал. Он будет заблокирован, если попытается повторно захватить ресурс, который удерживает.

#### С блокировками и без. Пример–сравнение
Что происходит, когда два потока бьются за ресурсы, и как при этом сохранить целостность данных?

Возьмём простейшие операции инкремента и декремента (увеличения и уменьшения числа). В роли общих ресурсов выступят глобальные числовые переменные: назовём их protected_resource и unprotected_resource. К каждой обратятся по два потока: один будет в цикле увеличивать значение с 0 до 500 000, другой — уменьшать до 0. Первую переменную обработаем с блокировками, а вторую — без.

In [9]:
import threading

protected_resource = 0
unprotected_resource = 0

NUM = 500000
mutex = threading.Lock()

# Потокобезопасный инкремент
def safe_plus():
    global protected_resource
    for i in range(NUM):
        # Ставим блокировку
        mutex.acquire()
        protected_resource += 1
        mutex.release()

# Потокобезопасный декремент
def safe_minus():
    global protected_resource
    for i in range(NUM):
        mutex.acquire()
        protected_resource -= 1
        mutex.release()

# То же, но без блокировки
def risky_plus():
    global unprotected_resource
    for i in range(NUM):
        unprotected_resource += 1

def risky_minus():
    global unprotected_resource
    for i in range(NUM):
        unprotected_resource -= 1

Создадим 4 потока, которые будут выполнять функции с блокировками и без:

In [12]:
thread1 = threading.Thread(target = safe_plus)
thread2 = threading.Thread(target = safe_minus)
thread3 = threading.Thread(target = risky_plus)
thread4 = threading.Thread(target = risky_minus)
thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread1.join()
thread2.join()
thread3.join()
thread4.join()
print ("Результат при работе с блокировкой %s" % protected_resource)
print ("Результат без блокировки %s" % unprotected_resource)

Результат при работе с блокировкой 0
Результат без блокировки 65998


Запускаем код несколько раз подряд и видим, что полученное без блокировки значение меняется случайным образом. При использовании блокировки всё работает последовательно: сначала значение растёт, затем — уменьшается, и в итоге получаем 0. А потоки thread3 и thread4 работают без блокировки и наперебой обращаются к глобальной переменной. Каждый выполняет столько операций своего цикла, сколько успевает за время активности. Поэтому при каждом запуске получаем случайные числа.

##### Как избежать взаимных блокировок?
Следите, чтобы у нескольких блокировок не было шанса сработать одновременно. Иначе одна заглушка перекроет один поток, другая — другой, и может случиться взаимная блокировка — тупик (deadlock). Это ситуация, когда ни один поток не имеет права действовать и программа зависает или рушится.

Если есть «захват» мьютекса, ничто не должно помешать последующему «высвобождению». Это значит, что release() должен срабатывать, как только блокировка становится не нужна.

Пишите код так, чтобы блокировки снимались, даже если функция выбрасывает исключение и завершает работу нештатно. Подстраховаться можно с помощью конструкции try-except-finally:

try:
    mutex.acquire()
    # Ваш код...

except SomethingGoesWrong:
    # Обрабатываем исключения

finally:
    # Ещё код
    mutex.release()